In [99]:
import os
import numpy as np
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Flatten
from sklearn.model_selection import train_test_split

In [100]:
# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(PARENT_DIRECTORY)

# Open the config file and load its content into a dictionary
config_file = open(PARENT_DIRECTORY + '\\config\\config.json')
CONFIG_DATA = json.load(config_file)

# Close the file after loading the data
config_file.close()

In [101]:
WINDOW_SIZE = CONFIG_DATA['WINDOW_SIZE']
NUMBER_OF_FEATURES = CONFIG_DATA["NUMBER_OF_FEATURES"]
NUM_EPOCHS = CONFIG_DATA["NUMBER_OF_EPOCHS"]
CLASSICATIONS = CONFIG_DATA['CLASSES_MOTIONSPEED']

ALL_X_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TRAINING_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

ALL_X_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'
ALL_Y_TEST_DATA_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTIONSPEED-SML-STEPS-V0.npy'

MOTION_MODEL_SAVE_PATH =  PARENT_DIRECTORY + '\\NeuralNetwork\\models\\standard-backprop-motionspeed-sml-steps-2.h5'

In [102]:
ALL_X_TRAIN_DATA = np.load(ALL_X_TRAINING_DATA_PATH)
ALL_Y_TRAIN_DATA = np.load(ALL_Y_TRAINING_DATA_PATH)

ALL_X_TEST_DATA = np.load(ALL_X_TEST_DATA_PATH)
ALL_Y_TEST_DATA = np.load(ALL_Y_TEST_DATA_PATH)

ALL_X_TRAIN_DATA = ALL_X_TRAIN_DATA[:, [0, 2, 4, 6]]
ALL_X_TEST_DATA = ALL_X_TEST_DATA[:, [0, 2, 4, 6]]

# Sum between columns 0 and 1
sum_cols_01 = np.sum(ALL_X_TRAIN_DATA[:, 0:2], axis=1)

# Sum between columns 2 and 3
sum_cols_23 = np.sum(ALL_X_TRAIN_DATA[:, 2:4], axis=1)

ALL_X_TRAIN_DATA =  np.column_stack((sum_cols_01, sum_cols_23))

# Sum between columns 0 and 1
sum_cols_01 = np.sum(ALL_X_TEST_DATA[:, 0:2], axis=1)
# Sum between columns 2 and 3
sum_cols_23 = np.sum(ALL_X_TEST_DATA[:, 2:4], axis=1)

ALL_X_TEST_DATA =  np.column_stack((sum_cols_01, sum_cols_23))

ALL_X_COMBINED_DATA = np.concatenate((ALL_X_TRAIN_DATA, ALL_X_TEST_DATA))
ALL_Y_COMBINED_DATA = np.concatenate((ALL_Y_TRAIN_DATA - 1, ALL_Y_TEST_DATA - 1))


ALL_X_TRAIN_DATA, ALL_X_TEST_DATA, ALL_Y_TRAIN_DATA, ALL_Y_TEST_DATA = train_test_split(ALL_X_COMBINED_DATA, ALL_Y_COMBINED_DATA, test_size=0.2, random_state=42)

print(ALL_Y_TRAIN_DATA[0])

print(ALL_X_TRAIN_DATA.shape)
print(ALL_Y_TRAIN_DATA.shape)

print(ALL_X_TEST_DATA.shape)
print(ALL_Y_TEST_DATA.shape)

0
(79726, 2)
(79726,)
(19932, 2)
(19932,)


In [103]:
# Generate random permutation indices
random_indices_1 = np.random.permutation(len(ALL_X_TRAIN_DATA))


# Shuffle the input features and target labels using the random indices
shuffled_X_TRAIN_DATA = ALL_X_TRAIN_DATA[random_indices_1]
shuffled_Y_TRAIN_DATA = ALL_Y_TRAIN_DATA[random_indices_1]

In [104]:
# Generate random permutation indices
random_indices_2 = np.random.permutation(len(ALL_X_TEST_DATA))

# Shuffle the input features and target labels using the random indices
shuffled_X_TEST_DATA = ALL_X_TEST_DATA[random_indices_2]
shuffled_Y_TEST_DATA = ALL_Y_TEST_DATA[random_indices_2]

In [105]:
output_activation_function = 'sigmoid' if len(CLASSICATIONS) == 3 else 'softmax'
loss_algorithm = 'binary_crossentropy' if len(CLASSICATIONS) == 3 else 'sparse_categorical_crossentropy'
output_size = 1 if len(CLASSICATIONS) == 3 else len(CLASSICATIONS)

In [106]:
model2_motion = Sequential()
model2_motion.add(InputLayer((2)))
model2_motion.add(Dense(8, activation='relu'))
model2_motion.add(Dense(4, activation='relu'))
model2_motion.add(Dense(output_size, activation=output_activation_function))

model2_motion.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 8)                 24        
                                                                 
 dense_31 (Dense)            (None, 4)                 36        
                                                                 
 dense_32 (Dense)            (None, 1)                 5         
                                                                 
Total params: 65
Trainable params: 65
Non-trainable params: 0
_________________________________________________________________


In [107]:
# Compile the model
model2_motion.compile(optimizer='adam', loss=loss_algorithm, metrics=['acc'])

history2_motion = model2_motion.fit(shuffled_X_TRAIN_DATA, shuffled_Y_TRAIN_DATA, validation_data=(shuffled_X_TEST_DATA, shuffled_Y_TEST_DATA), epochs=NUM_EPOCHS)

# Save the model to a file
model2_motion.save(MOTION_MODEL_SAVE_PATH)

Epoch 1/12


2492/2492 [==============================] - 6s 2ms/step - loss: 0.4696 - acc: 0.8161 - val_loss: 0.2834 - val_acc: 0.9070
Epoch 2/12
2492/2492 [==============================] - 5s 2ms/step - loss: 0.2592 - acc: 0.9039 - val_loss: 0.2400 - val_acc: 0.9089
Epoch 3/12
2492/2492 [==============================] - 5s 2ms/step - loss: 0.2370 - acc: 0.9048 - val_loss: 0.2375 - val_acc: 0.9048
Epoch 4/12
2492/2492 [==============================] - 5s 2ms/step - loss: 0.2316 - acc: 0.9043 - val_loss: 0.2258 - val_acc: 0.9052
Epoch 5/12
2492/2492 [==============================] - 4s 2ms/step - loss: 0.2283 - acc: 0.9044 - val_loss: 0.2215 - val_acc: 0.9078
Epoch 6/12
2492/2492 [==============================] - 4s 2ms/step - loss: 0.2257 - acc: 0.9042 - val_loss: 0.2275 - val_acc: 0.9006
Epoch 7/12
2492/2492 [==============================] - 4s 2ms/step - loss: 0.2252 - acc: 0.9041 - val_loss: 0.2193 - val_acc: 0.9075
Epoch 8/12
2492/2492 [==============================] - 4s 2ms/step - los